### Limma voom analysis of Age Dataset: RNA-seq


In [35]:
rm(list=ls())
#load necessary libraries 
library(ggplot2)
library("BiocParallel")
parallelFlag=TRUE
register(MulticoreParam(50))
library(sva)
library(limma)
library(statmod)

## Load data and design

In [36]:
##load RNA-seq TPM
data=read.table('../age.rna.counts.txt.tpm',header=TRUE,sep='\t')
rownames(data)=paste(data$GeneID,data$GeneName,sep="_")
data$GeneID=NULL
data$GeneName=NULL

In [37]:


data=data[rowSums(data)>0,]

head(data)

,d0_A_Rep1,d0_A_Rep2,d0_A_Rep3,d0_A_Rep4,d0_Y_Pax7_Rep1,d0_Y_Pax7_Rep2,d0_Y_Pax7_Rep3,d0_Y_Rep1,d0_Y_Rep2,d0_Y_Rep3,⋯,d5_Y_Rep2,d5_Y_Rep3,d7_A_Rep1,d7_A_Rep2,d7_A_Rep3,d7_A_Rep4,d7_Y_Rep1,d7_Y_Rep2,d7_Y_Rep3,d7_Y_Rep4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000000001_Gnai3,33.66,32.74,138.68,103.55,162.42,146.37,119.82,109.23,157.71,63.06,⋯,159.77,121.53,183.29,175.34,156.22,164.38,120.19,132.60,175.81,191.01
ENSMUSG00000000028_Cdc45,1.90,2.33,3.65,8.25,5.82,2.95,2.78,4.28,7.65,6.26,⋯,26.31,28.41,22.41,23.37,21.64,14.82,31.30,12.53,36.95,31.72
ENSMUSG00000000031_H19,1.72,2.05,5.48,5.61,35.36,5.57,2.34,1.06,0.66,4.91,⋯,75.14,161.20,58.76,66.95,42.86,35.30,71.70,34.19,25.50,45.81
ENSMUSG00000000037_Scml2,0.39,0.20,0.82,1.77,1.54,0.77,0.40,0.25,1.02,0.50,⋯,2.56,2.72,2.86,2.26,4.80,4.08,1.39,1.37,3.17,0.53
ENSMUSG00000000049_Apoh,1.22,1.11,0.00,0.35,0.00,0.00,0.00,0.56,0.00,0.12,⋯,0.04,0.00,0.00,0.00,0.00,0.49,0.10,0.00,0.00,0.15
ENSMUSG00000000056_Narf,3.18,2.53,18.66,11.92,10.70,16.04,20.35,5.96,10.01,10.31,⋯,9.64,11.29,14.16,12.11,13.64,8.11,5.36,14.06,10.36,4.68


In [38]:
nrow(data)

[1] 37302

In [41]:
#load the metadata
batches=read.table("../batches.txt",header=TRUE,sep='\t')
batches$Batch=factor(batches$Batch)
batches$Pax7=factor(batches$Pax7)
batches$Age=factor(batches$Age)
batches$File=NULL
batches

Day,Age,Pax7,Batch,Rep
<int>,<fct>,<fct>,<fct>,<fct>
0,A,0,R2200,d0_A_Rep1
0,A,0,R2081,d0_A_Rep2
0,A,0,R2830,d0_A_Rep3
0,A,0,R2830,d0_A_Rep4
0,Y,1,R2879,d0_Y_Pax7_Rep1
0,Y,1,R2879,d0_Y_Pax7_Rep2
0,Y,1,R2879,d0_Y_Pax7_Rep3
0,Y,0,R2081,d0_Y_Rep1
0,Y,0,R2081,d0_Y_Rep2


In [66]:
#use asinh(tpm)
E=as.matrix(asinh(data))

## Correlation 

In [67]:
spearman_cor=cor(E, method = "spearman")

In [68]:
pearson_cor=cor(E,method="pearson")

In [69]:
write.table(spearman_cor,file="rna.uncorrected.spearman_r.tsv",quote=FALSE)
write.table(pearson_cor,file="rna.uncorrected.pearson_r.tsv",quote=FALSE)

In [73]:
## plot correlation heatmaps 
library(gplots)
require(gtools)
require(RColorBrewer)
cols <- colorRampPalette(brewer.pal(10, "RdBu"))(256)

svg(filename="RNA_rep_spearman_cor_preSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(spearman_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(10,10),
          main="RNA-seq replicates spearman correlation -- pre-SVA")
dev.off() 


png 
  2

In [74]:
dev.off() 

null device 
          1

In [75]:
svg(filename="RNA_rep_pearson_cor_preSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(pearson_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(10,10),          
          main="RNA-seq replicates pearson correlation -- pre-SVA")

dev.off() 

png 
  2

## Run PCA 

In [48]:
data.pca=prcomp(t(E))

In [49]:
var_explained=as.character(round(100*data.pca$sdev^2/sum(data.pca$sdev^2),2))

In [76]:
svg(filename="age_rna_uncorrected_scree.svg",
   height=3,
   width=4,
   pointsize=12)
print(barplot(100*data.pca$sdev^2/sum(data.pca$sdev^2),las=2,ylab="% Variance Explained",xlab="Principal Component",ylim=c(0,40), xlim=c(0,10)))
dev.off() 

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5
[11,] 12.7
[12,] 13.9
[13,] 15.1
[14,] 16.3
[15,] 17.5
[16,] 18.7
[17,] 19.9
[18,] 21.1
[19,] 22.3
[20,] 23.5
[21,] 24.7
[22,] 25.9
[23,] 27.1
[24,] 28.3
[25,] 29.5
[26,] 30.7
[27,] 31.9
[28,] 33.1
[29,] 34.3
[30,] 35.5
[31,] 36.7
[32,] 37.9
[33,] 39.1
[34,] 40.3
[35,] 41.5
[36,] 42.7
[37,] 43.9
[38,] 45.1
[39,] 46.3
[40,] 47.5
[41,] 48.7


png 
  2

In [77]:
pca_df=data.frame(data.pca$x)
pca_df=cbind(pca_df,batches)
pca_df$Day=factor(pca_df$Day)

In [78]:
svg(filename="age_rna_uncorrected_PC1_vs_PC2_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC2,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#000000'))+
theme_bw() )
dev.off() 

png 
  2

In [79]:
svg(filename="age_rna_uncorrected_PC2_vs_PC3_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC2,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#000000'))+
theme_bw())
dev.off() 

png 
  2

In [80]:
svg(filename="age_rna_uncorrected_PC1_vs_PC3_batch.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00','#000000'))+
theme_bw())
dev.off() 

png 
  2

## Running SVA correction 

In [81]:
head(batches)

Day,Age,Pax7,Batch,Rep,Grouping
<int>,<fct>,<fct>,<fct>,<fct>,<fct>
0,A,0,R2200,d0_A_Rep1,0.A.0
0,A,0,R2081,d0_A_Rep2,0.A.0
0,A,0,R2830,d0_A_Rep3,0.A.0
0,A,0,R2830,d0_A_Rep4,0.A.0
0,Y,1,R2879,d0_Y_Pax7_Rep1,0.Y.1
0,Y,1,R2879,d0_Y_Pax7_Rep2,0.Y.1


In [82]:
Grouping <- factor(paste0(batches$Day,".",batches$Age, ".", batches$Pax7))
batches$Grouping=Grouping

In [83]:
mod0=model.matrix(~1,data=batches)
#mod1=model.matrix(~Day+Age+Pax7+Batch,data=batches)
mod1=model.matrix(~0+Grouping+Batch,data=batches)

In [84]:
sva.obj=sva(E,mod1,mod0)

Number of significant surrogate variables is:  7 
Iteration (out of 5 ):1  2  3  4  5  

In [85]:
sur_var=data.frame(sva.obj$sv)

In [86]:
full.design.sv=cbind(mod1,sur_var)


In [87]:
#save the full design so we don't have to run SVA next time 
write.table(full.design.sv,file="full.design.sv.txt",quote=FALSE,sep='\t')


In [88]:
#full.design.sv=read.table("full.design.sv.txt",header=TRUE,sep='\t')


### Perform pca on the "cleaned" matrix with surrogate variable contributions removed


In [89]:
nrow(batches)

[1] 41

In [90]:
cleaned_E=removeBatchEffect(E,batch=batches$Batch,covariates=sur_var,design=model.matrix(~0+Grouping,data=batches))


In [91]:
## Run replicate correlation on SVA corrected dataset 
spearman_cor=cor(cleaned_E, method = "spearman")
pearson_cor=cor(cleaned_E,method="pearson")

In [92]:
write.table(spearman_cor,file="rna.corrected.spearman_r.tsv",quote=FALSE)
write.table(pearson_cor,file="rna.corrected.pearson_r.tsv",quote=FALSE)

In [93]:

svg(filename="RNA_rep_spearman_cor_postSVA.svg",
   height=8,
   width=8,
   pointsize=12)
cur_h=heatmap.2(as.matrix(spearman_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(12,12),          
          main="RNA-seq replicates spearman correlation -- post-SVA ")
dev.off() 



png 
  2

In [94]:
svg(filename="RNA_rep_pearson_cor_postSVA.svg",
   height=8,
   width=8,
   pointsize=12)

cur_h=heatmap.2(as.matrix(pearson_cor), 
          trace="none", 
          scale="none", 
          Rowv=TRUE,
          Colv=TRUE,
          col=rev(cols), 
          dendrogram="none",
          margins=c(12,12),          
          main="RNA-seq replicates pearson correlation -- post SVA")
dev.off() 


png 
  2

In [95]:
data.pca=prcomp(t(cleaned_E))

In [96]:
var_explained=as.character(round(100*data.pca$sdev^2/sum(data.pca$sdev^2),2))


In [97]:
svg(filename="age_rna_corrected_scree.svg",
   height=3,
   width=4,
   pointsize=12)
print(barplot(100*data.pca$sdev^2/sum(data.pca$sdev^2),las=2,ylab="% Variance Explained",xlab="Principal Component",ylim=c(0,40), xlim=c(0,10)))
dev.off()

      [,1]
 [1,]  0.7
 [2,]  1.9
 [3,]  3.1
 [4,]  4.3
 [5,]  5.5
 [6,]  6.7
 [7,]  7.9
 [8,]  9.1
 [9,] 10.3
[10,] 11.5
[11,] 12.7
[12,] 13.9
[13,] 15.1
[14,] 16.3
[15,] 17.5
[16,] 18.7
[17,] 19.9
[18,] 21.1
[19,] 22.3
[20,] 23.5
[21,] 24.7
[22,] 25.9
[23,] 27.1
[24,] 28.3
[25,] 29.5
[26,] 30.7
[27,] 31.9
[28,] 33.1
[29,] 34.3
[30,] 35.5
[31,] 36.7
[32,] 37.9
[33,] 39.1
[34,] 40.3
[35,] 41.5
[36,] 42.7
[37,] 43.9
[38,] 45.1
[39,] 46.3
[40,] 47.5
[41,] 48.7


png 
  2

In [98]:
var_explained[0:10]


[1] "39.21" "13.96" "9.38"  "6.23"  "5.37"  "3.69"  "3.15"  "2.8"   "1.94" 
[10] "1.47"

In [99]:
pca_df=data.frame(data.pca$x)
pca_df=cbind(pca_df,batches)
pca_df$Day=factor(pca_df$Day)

In [100]:
svg(filename="age_rna_corrected_PC1_vs_PC2.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC2,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00'))+
theme_bw())
dev.off()

png 
  2

In [101]:
svg(filename="age_rna_corrected_PC2_vs_PC3.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC2,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00'))+
theme_bw())
dev.off()

png 
  2

In [102]:
svg(filename="age_rna_corrected_PC1_vs_PC3.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=pca_df,aes(x=pca_df$PC1,y=pca_df$PC3,color=pca_df$Day,shape=pca_df$Age,label=pca_df$Rep))+
geom_point(size=5)+
geom_text(nudge_x=2,nudge_y = 12,size=2)+
scale_color_manual(values=c('#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00'))+
theme_bw())
dev.off()

png 
  2

### fit model with limma 

In [103]:
#fit <- lmFit(E,full.design.sv)
fit = lmFit(cleaned_E,model.matrix(~0+Grouping))

In [104]:
colnames(fit)


[1] "Grouping0.A.0" "Grouping0.Y.0" "Grouping0.Y.1" "Grouping1.A.0"
 [5] "Grouping1.Y.0" "Grouping3.A.0" "Grouping3.Y.0" "Grouping5.A.0"
 [9] "Grouping5.Y.0" "Grouping7.A.0" "Grouping7.Y.0"

In [105]:
colnames(fit$coefficients)

[1] "Grouping0.A.0" "Grouping0.Y.0" "Grouping0.Y.1" "Grouping1.A.0"
 [5] "Grouping1.Y.0" "Grouping3.A.0" "Grouping3.Y.0" "Grouping5.A.0"
 [9] "Grouping5.Y.0" "Grouping7.A.0" "Grouping7.Y.0"

###  Contrasts 

In [106]:
#create contrasts of interest 
cont.matrix=makeContrasts(
    d0_Y_vs_A="Grouping0.Y.0 - Grouping0.A.0",  
    d1_Y_vs_A="Grouping1.Y.0 - Grouping1.A.0",
    d3_Y_vs_A="Grouping3.Y.0 - Grouping3.A.0",
    d5_Y_vs_A="Grouping5.Y.0 - Grouping5.A.0",
    d7_Y_vs_A="Grouping7.Y.0 - Grouping7.A.0",
    d1_Y_vs_d0_Y="Grouping1.Y.0 - Grouping0.Y.0",
    d1_A_vs_d0_A="Grouping1.A.0 - Grouping0.A.0",
    d3_Y_vs_d1_Y="Grouping3.Y.0 - Grouping1.Y.0",
    d3_A_vs_d1_A="Grouping3.A.0 - Grouping1.A.0",
    d5_Y_vs_d3_Y="Grouping5.Y.0 - Grouping3.Y.0",
    d5_A_vs_d3_A="Grouping5.A.0 - Grouping3.A.0",
    d7_Y_vs_d5_Y="Grouping7.Y.0 - Grouping5.Y.0",
    d7_A_vs_d5_A="Grouping7.A.0 - Grouping5.A.0",
    d0_Y_Pax_7_vs_d0_Y_noPax7="Grouping0.Y.1 - Grouping0.Y.0",
    levels=model.matrix(~0+Grouping))


In [107]:
pval_thresh=0.01
lfc_thresh=1

In [108]:
fit2=contrasts.fit(fit,cont.matrix)
e=eBayes(fit2)
comparisons=colnames(cont.matrix)

In [109]:
for(i in seq(1,length(comparisons)))
{
  tab<-topTable(e, number=nrow(e),coef=i,lfc=lfc_thresh, p.value = pval_thresh)
  up=sum(tab$logFC>0)
  down=sum(tab$logFC<0)
  sig=nrow(tab)
  curtitle=paste(comparisons[i],'\n','sig:',sig,'\n','up:',up,'\n','down:',down,'\n')
  print(curtitle)
  vals=topTable(e,number=nrow(e),coef=i)
  vals$pscaled=-1*log10(vals$adj.P.Val)
  vals$sig=vals$adj.P.Val<pval_thresh & abs(vals$logFC)>lfc_thresh 
  png(paste("volcano_diff",comparisons[i],".png",sep=""))
  print(ggplot(data=vals,
               aes(y=vals$pscaled,x=vals$logFC,color=vals$sig))+
               geom_point(alpha=0.1)+
               xlab("log2(FC)")+
               ylab("-log10(pval)")+
               ggtitle(curtitle)+
               theme_bw()+
               scale_color_manual(values=c("#000000","#FF0000")))
  dev.off() 
  write.table(tab,file=paste("diff_",comparisons[i],".tsv",sep=""),quote=FALSE,sep='\t',row.names = TRUE,col.names = TRUE)
}
    

[1] "d0_Y_vs_A \n sig: 565 \n up: 365 \n down: 200 \n"
[1] "d1_Y_vs_A \n sig: 1629 \n up: 494 \n down: 1135 \n"
[1] "d3_Y_vs_A \n sig: 57 \n up: 14 \n down: 43 \n"
[1] "d5_Y_vs_A \n sig: 2643 \n up: 1726 \n down: 917 \n"
[1] "d7_Y_vs_A \n sig: 848 \n up: 505 \n down: 343 \n"
[1] "d1_Y_vs_d0_Y \n sig: 626 \n up: 434 \n down: 192 \n"
[1] "d1_A_vs_d0_A \n sig: 3515 \n up: 3171 \n down: 344 \n"
[1] "d3_Y_vs_d1_Y \n sig: 2395 \n up: 1401 \n down: 994 \n"
[1] "d3_A_vs_d1_A \n sig: 1060 \n up: 389 \n down: 671 \n"
[1] "d5_Y_vs_d3_Y \n sig: 944 \n up: 504 \n down: 440 \n"
[1] "d5_A_vs_d3_A \n sig: 3400 \n up: 1074 \n down: 2326 \n"
[1] "d7_Y_vs_d5_Y \n sig: 1040 \n up: 545 \n down: 495 \n"
[1] "d7_A_vs_d5_A \n sig: 2053 \n up: 1248 \n down: 805 \n"
[1] "d0_Y_Pax_7_vs_d0_Y_noPax7 \n sig: 2602 \n up: 1856 \n down: 746 \n"


In [113]:
cleaned_E_df=as.data.frame(cleaned_E)
head(cleaned_E_df)

,d0_A_Rep1,d0_A_Rep2,d0_A_Rep3,d0_A_Rep4,d0_Y_Pax7_Rep1,d0_Y_Pax7_Rep2,d0_Y_Pax7_Rep3,d0_Y_Rep1,d0_Y_Rep2,d0_Y_Rep3,⋯,d5_Y_Rep2,d5_Y_Rep3,d7_A_Rep1,d7_A_Rep2,d7_A_Rep3,d7_A_Rep4,d7_Y_Rep1,d7_Y_Rep2,d7_Y_Rep3,d7_Y_Rep4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000000001_Gnai3,4.7997213,5.1373116,5.2436664,4.8343823,4.77515815,4.89767452,4.73578084,5.6285891,5.633120010,5.5384033,⋯,5.870667376,5.8141455,5.6033929,5.5280285,5.607970481,5.5027761,5.556444772,5.424664,5.58144392,5.4482103
ENSMUSG00000000028_Cdc45,2.1224989,2.0315831,1.9388268,2.6210054,1.80975407,2.03398878,2.03381833,2.0283776,2.708310785,2.7331228,⋯,3.963019613,3.7451557,3.6649362,3.7410577,3.630358077,3.3307439,4.286974490,3.569443,4.20931391,3.6340675
ENSMUSG00000000031_H19,2.3771394,1.7311036,2.2454405,2.0734960,4.51653317,4.95319289,4.41803069,0.5930675,1.791865280,2.1539748,⋯,4.427044590,4.3248902,3.8563617,4.2577650,3.750626402,3.9572555,4.885033715,5.001598,4.85815815,4.5428810
ENSMUSG00000000037_Scml2,0.3620911,0.2767164,0.0658098,0.5991354,0.06495702,0.35692941,-0.04384437,0.2960269,0.708819114,0.7135996,⋯,2.139982970,2.1201185,2.1105548,1.9081506,2.003212367,1.5654627,1.443000091,1.416933,1.78097784,1.1795386
ENSMUSG00000000049_Apoh,0.6239051,0.6633686,0.2273006,0.6485407,0.03622449,-0.09899279,-0.07571410,0.4958954,0.001118024,0.1206518,⋯,-0.003498623,0.1149913,0.2026911,0.3283521,0.006527713,0.5186474,0.007997031,0.104385,-0.06611364,0.3064039
ENSMUSG00000000056_Narf,2.2949361,2.1674840,2.7829372,2.1176154,2.50770420,2.81541258,2.64287572,2.5159142,2.964207172,2.8615585,⋯,3.255374805,3.3323751,3.1037782,3.0771856,3.700265710,2.7580200,2.749055881,2.654638,2.89294476,2.4129904


## Plot d0_Y vs d0_Y_Pax7

In [114]:
d0_Y=(cleaned_E_df$d0_Y_Rep1+
    cleaned_E_df$d0_Y_Rep2+
    cleaned_E_df$d0_Y_Rep3+
    cleaned_E_df$d0_Y_Rep4+
    cleaned_E_df$d0_Y_Rep5+
    cleaned_E_df$d0_Y_Rep6)/6
d0_Y_Pax7=(cleaned_E_df$d0_Y_Pax7_Rep1+
          cleaned_E_df$d0_Y_Pax7_Rep2+
          cleaned_E_df$d0_Y_Pax7_Rep3)/3

In [125]:
d0_Y_df=data.frame(rownames(cleaned_E_df),d0_Y,d0_Y_Pax7)

In [126]:
head(d0_Y_df)

rownames.cleaned_E_df.,d0_Y,d0_Y_Pax7
<fct>,<dbl>,<dbl>
ENSMUSG00000000001_Gnai3,5.5635971,4.8028712
ENSMUSG00000000028_Cdc45,2.4758636,1.9591871
ENSMUSG00000000031_H19,1.6292993,4.6292522
ENSMUSG00000000037_Scml2,0.5518297,0.1260140
ENSMUSG00000000049_Apoh,0.2119634,-0.0461608
ENSMUSG00000000056_Narf,2.7657139,2.6553308


In [137]:
svg(filename="RNA_d0Y_vs_d0YPax7.svg",
   height=8,
   width=8,
   pointsize=12)
print(ggplot(data=d0_Y_df,
      aes(x=d0_Y_df$d0_Y,
         y=d0_Y_df$d0_Y_Pax7))+
geom_point(alpha=0.3)+
geom_abline(slope=1)+
xlab("D0 Young")+
ylab("D0 Young PAX7")+
ggtitle("asinh(TPM) RNA expression")+
theme_bw(20))
dev.off()

png 
  2

In [133]:
d0_Y_df$delta=d0_Y_df$d0_Y_Pax7/d0_Y_df$d0_Y

In [134]:
d0_Y_df=d0_Y_df[order(d0_Y_df$delta),]

In [135]:
head(d0_Y_df)

,rownames.cleaned_E_df.,d0_Y,d0_Y_Pax7,delta
,<fct>,<dbl>,<dbl>,<dbl>
34478,ENSMUSG00000107604_Gm44041,-1.555546e-05,0.24682828,-15867.6271
17939,ENSMUSG00000067610_Klri1,-2.436050e-04,0.81214533,-3333.8613
381,ENSMUSG00000002190_Clgn,-2.079709e-05,0.03072402,-1477.3231
25368,ENSMUSG00000089598_Gm26158,7.104888e-04,-1.04954682,-1477.2180
32023,ENSMUSG00000103874_Gm38360,-2.277092e-04,0.23237066,-1020.4713
28335,ENSMUSG00000096924_Gm26824,2.093844e-04,-0.18243210,-871.2782


In [136]:
tail(d0_Y_df)

,rownames.cleaned_E_df.,d0_Y,d0_Y_Pax7,delta
,<fct>,<dbl>,<dbl>,<dbl>
32772,ENSMUSG00000105025_Rnu3b1,8.611725e-04,1.16838425,1356.737
6384,ENSMUSG00000028071_Sh2d2a,3.227369e-04,0.82993165,2571.542
35739,ENSMUSG00000109467_Gm30646,1.929916e-05,0.07070968,3663.874
20297,ENSMUSG00000078945_Naip2,7.460942e-05,0.36685790,4917.045
10360,ENSMUSG00000036925_Mucl2,1.049093e-05,0.11567733,11026.415
13191,ENSMUSG00000045440_Insm2,-3.336446e-06,-0.08638471,25891.236


## Ddit3

In [147]:
ddit3_subset=sinh(cleaned_E_df[rownames(cleaned_E_df)=="ENSMUSG00000025408_Ddit3",])

In [155]:
write.table(ddit3_subset,file="DDIT3_tpm_svacorrected.txt")